In [ ]:
import json
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer

In [ ]:
save_path = "save_path"
loaded_model = AutoModelForSequenceClassification.from_pretrained(save_path)
loaded_tokenizer = AutoTokenizer.from_pretrained(save_path)

print(f"Model and tokenizer loaded from {save_path}")

In [4]:
from datasets import Dataset, DatasetDict
raw_datasets = DatasetDict({
    "train": Dataset.from_csv("train.csv"),
    "eval": Dataset.from_csv("eval.csv")
})

In [ ]:
tokenized_datasets = raw_datasets.map(lambda dataset: loaded_tokenizer(dataset['textual_embedding'], truncation=True, padding=True), batched=True)

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["textual_embedding"])
tokenized_datasets = tokenized_datasets.rename_column("label_int", "labels")

In [ ]:
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
import numpy as np
import evaluate
from sklearn.metrics import classification_report


data_collator = DataCollatorWithPadding(tokenizer=loaded_tokenizer)

training_args = TrainingArguments("test-trainer", num_train_epochs=1, evaluation_strategy="epoch", 
                                  weight_decay=5e-4, save_strategy="no", report_to="none")


# Metric for validation error
def compute_metrics(eval_preds):
    metric = evaluate.load("accuracy") # F1 and Accuracy
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Evaluate the loaded model
trainer = Trainer(
    model=loaded_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"],
    data_collator=data_collator,
    tokenizer=loaded_tokenizer,
    compute_metrics=compute_metrics
)


eval_result = trainer.evaluate()


print(f"Evaluation Results: {eval_result}")
print(f"Evaluation Loss: {eval_result['eval_loss']:.4f}")

y_pred = trainer.predict(tokenized_datasets["eval"]).predictions
y_pred = np.argmax(y_pred, axis=-1)

y_true = tokenized_datasets["eval"]["labels"]
print(classification_report(y_true, y_pred, digits=3))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np


labels = [
    "Normal", "MITM", "Uploading", "Ransomware", "SQL_injection",
    "DDoS_HTTP", "DDoS_TCP", "Password", "Port_Scanning", "Vulnerability_scanner",
    "Backdoor", "XSS", "Fingerprinting", "DDoS_UDP", "DDoS_ICMP"
]


cf_matrix = confusion_matrix(y_true, y_pred)


cf_matrix_percent = cf_matrix / cf_matrix.sum(axis=1)[:, np.newaxis] * 100


plt.figure(figsize=(10, 10))
sns.heatmap(cf_matrix_percent, annot=True, fmt=".2f", cmap="Blues",
            xticklabels=labels, yticklabels=labels)


plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix with Percentages")
plt.xticks(rotation=45, ha='right')  
plt.yticks(rotation=0)              
plt.show()
